In [2]:
library(data.table)
library(lmtest)
library(broom)

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



In [8]:
n <- 1000
d <- data.table(age=sample(18:39, n, replace=T), edu=sample(1:3, n, replace=T), treat=sample(0:1, n, replace=T))
d$edu <- as.factor(d$edu)
d <- d[, y:=age*1000 + treat * 5000 + rnorm(n, mean=0, sd=100)]
d <- d[, y:=y+(edu==1)*50*treat]
d <- d[, y:=y+(edu==2)*250*treat]
d <- d[, y:=y+(edu==3)*1000*treat]

In [9]:
m <- lm(y ~ age + treat + treat:edu, data = d)
coeftest(m)


t test of coefficients:

              Estimate Std. Error   t value Pr(>|t|)    
(Intercept)   -1.11802   15.56648   -0.0718   0.9428    
age         1000.01917    0.51735 1932.9749   <2e-16 ***
treat       5049.22623    9.40865  536.6580   <2e-16 ***
treat:edu2   210.62010   11.49847   18.3172   <2e-16 ***
treat:edu3   948.86964   11.37432   83.4221   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [10]:
coefs <- data.table(tidy(coeftest(m)))


In [11]:
# Effect of 'treat' when edu == 1
# Should be 5000 + 50
baseline <- coefs[3,estimate]
baseline

[1] 5049.226

In [12]:
# Effect of 'treat' when edu == 2
# Should be 5000 + 250

# the treat:edu2 coefficients means the effect of treat+edu2 when edu is changed from 1 to 2.
coefs[4,estimate] + baseline 

[1] 5259.846

In [13]:
# Effect of 'treat' when edu == 3
# Should be 5000 + 1000
coefs[5,estimate] + baseline 

[1] 5998.096

### Try the method of creating one dummy for each factor level

In [30]:
d<-d[,edu1 := as.integer(edu=="1")]
d<-d[,edu2 := as.integer(edu=="2")]
d<-d[,edu3 := as.integer(edu=="3")]

In [31]:
m1 <- lm(y~age + treat:edu1 + treat:edu2 + treat:edu3, data=d)

In [32]:
coeftest(m1)


t test of coefficients:

              Estimate Std. Error   t value Pr(>|t|)    
(Intercept)   -1.11802   15.56648   -0.0718   0.9428    
age         1000.01917    0.51735 1932.9749   <2e-16 ***
treat:edu1  5049.22623    9.40865  536.6580   <2e-16 ***
treat:edu2  5259.84633    9.29644  565.7914   <2e-16 ***
treat:edu3  5998.09588    9.13660  656.4910   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### The results are pretty close to the cases where we do lm on subset of data?

In [14]:
coeftest(lm(y ~ age + treat, data = d[edu=='1']))
coeftest(lm(y ~ age + treat, data = d[edu=='2']))
coeftest(lm(y ~ age + treat, data = d[edu=='3']))


t test of coefficients:

              Estimate Std. Error  t value Pr(>|t|)    
(Intercept)   14.46697   27.39859    0.528   0.5978    
age          999.25905    0.90386 1105.548   <2e-16 ***
treat       5055.86539   11.10399  455.320   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



t test of coefficients:

              Estimate Std. Error   t value Pr(>|t|)    
(Intercept)    0.56444   27.68411    0.0204   0.9837    
age         1000.50809    0.92311 1083.8395   <2e-16 ***
treat       5244.16311   11.88100  441.3907   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



t test of coefficients:

              Estimate Std. Error   t value Pr(>|t|)    
(Intercept)  -16.81643   25.78171   -0.6523   0.5147    
age         1000.29357    0.86125 1161.4384   <2e-16 ***
treat       6006.04464   11.07316  542.3967   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Use the multcomp package to calculate the combined effect and SE's of treat with a specific edu level

In [27]:
library(multcomp)

# See https://rpubs.com/djcava/lincom


Loading required package: mvtnorm
Loading required package: survival
Loading required package: TH.data
Loading required package: MASS

Attaching package: ‘TH.data’

The following object is masked from ‘package:MASS’:

    geyser



In [28]:
names(coef(m))  #Extract the coefficient names (b0,b1,b2,b3)


[1] "(Intercept)" "x"           "treat"       "treat:edu2"  "treat:edu3"

In [30]:
m.lh <- glht(m, linfct = c("treat + treat:edu2 = 0"))
summary(m.lh)



	 Simultaneous Tests for General Linear Hypotheses

Fit: lm(formula = y ~ x + treat + treat:edu, data = d)

Linear Hypotheses:
                        Estimate Std. Error t value Pr(>|t|)    
treat + treat:edu2 == 0  5256.42      23.87   220.2   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Adjusted p values reported -- single-step method)


In [31]:
confint(m.lh)


	 Simultaneous Confidence Intervals

Fit: lm(formula = y ~ x + treat + treat:edu, data = d)

Quantile = 1.9853
95% family-wise confidence level
 

Linear Hypotheses:
                        Estimate  lwr       upr      
treat + treat:edu2 == 0 5256.4205 5209.0244 5303.8167
